In [9]:
import subprocess
import sys

packages = [
    "numpy>=1.24.4",
    "opencv-python>=4.8.1",
    "pillow>=10.0.0",
    "requests>=2.31.0",
    "imutils>=0.5.4",
    "ultralytics>=8.0.196",
    "transformers>=4.33.2", 
    "torch>=2.0.1",
    "torchvision>=0.15.2",
    "easyocr>=1.6.2",
    "huggingface-hub>=0.16.4" 
]

for package in packages:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ Successfully installed {package}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install {package}: {e}")


✅ Successfully installed numpy>=1.24.4
✅ Successfully installed opencv-python>=4.8.1
✅ Successfully installed pillow>=10.0.0
✅ Successfully installed requests>=2.31.0
✅ Successfully installed imutils>=0.5.4
✅ Successfully installed ultralytics>=8.0.196
✅ Successfully installed transformers>=4.33.2
✅ Successfully installed torch>=2.0.1
✅ Successfully installed torchvision>=0.15.2
✅ Successfully installed easyocr>=1.6.2
✅ Successfully installed huggingface-hub>=0.16.4


In [5]:
import cv2
import numpy as np
import os
import threading
from queue import Queue
import time
from ultralytics import YOLO
YOLO_AVAILABLE = True

os.environ['TORCH_SERIALIZATION_WEIGHTS_ONLY'] = 'False'
os.environ['OMP_NUM_THREADS'] = '4'
cv2.setNumThreads(4)

class StreamSafe:
    def __init__(self):
        self.camera_index = None
        self.cap = None
        self.running = False
        self.frame_queue = Queue(maxsize=1)  
        self.processing_started = threading.Event()
        
        if YOLO_AVAILABLE:
            try:
                self.license_plate_model = YOLO('license-plate-finetune-v1l.pt')
                print("✅ License plate model loaded")
            except Exception as e:
                print(f"❌ Model loading failed: {e}")
                self.license_plate_model = None
        else:
            self.license_plate_model = None

    def blur_license_plates(self, frame):
        """Detect and blur vehicle license plates"""
        if self.license_plate_model is None:
            return frame
            
        try:
            results = self.license_plate_model.predict(
                source=frame,
                device='cpu',
                verbose=False,
                conf=0.65,
                imgsz=416,
                half=False,
                max_det=3,
                agnostic_nms=True
            )
            
            if len(results) > 0 and results[0].boxes is not None:
                boxes = results[0].boxes.xyxy.cpu().numpy()
                confidences = results[0].boxes.conf.cpu().numpy()
                
                for i, box in enumerate(boxes):
                    if confidences[i] > 0.65:
                        x1, y1, x2, y2 = map(int, box)
                        
                        x1, y1 = max(0, x1), max(0, y1)
                        x2 = min(x2, frame.shape[1])
                        y2 = min(y2, frame.shape[0])
                        
                        if x2 > x1 and y2 > y1:
                            plate_roi = frame[y1:y2, x1:x2]
                            if plate_roi.size > 0:
                                blurred_plate = cv2.GaussianBlur(plate_roi, (51, 51), 0)
                                frame[y1:y2, x1:x2] = blurred_plate
                    
        except Exception as e:
            pass
        
        return frame

    def blur_address_numbers(self, frame):
        """Detect and blur house numbers, unit numbers, block numbers"""
        # TODO: Implement address number detection and blurring
        # Example implementation:
        # - Use OCR model to detect text regions
        # - Filter for numeric patterns (addresses)
        # - Apply Gaussian blur to detected regions
        return frame

    def blur_street_signs(self, frame):
        """Detect and blur street signs showing road names"""
        # TODO: Implement street sign detection and blurring
        # Example implementation:
        # - Use YOLO model trained on street signs
        # - Detect rectangular sign shapes
        # - Apply blur to sign regions
        return frame

    def blur_screens(self, frame):
        """Detect and blur computer/phone screens"""
        # TODO: Implement screen detection and blurring
        # Example implementation:
        # - Use YOLO model trained on electronic screens
        # - Detect rectangular screen regions
        # - Apply heavy blur to screen content
        return frame

    def blur_documents(self, frame):
        """Detect and blur documents and papers"""
        # TODO: Implement document detection and blurring
        # Example implementation:
        # - Use YOLO model trained on paper documents
        # - Detect paper/document shapes
        # - Apply blur to document regions
        return frame

    def apply_privacy_protection(self, frame):
        """Apply all privacy protection methods to frame"""
        protected_frame = frame.copy()
        
        protected_frame = self.blur_license_plates(protected_frame)
        protected_frame = self.blur_address_numbers(protected_frame)
        protected_frame = self.blur_street_signs(protected_frame)
        protected_frame = self.blur_screens(protected_frame)
        protected_frame = self.blur_documents(protected_frame)
        
        return protected_frame

    def instant_frame_capture_thread(self):
        """Ultra-fast frame capture for instant processing"""
        while self.running:
            ret, frame = self.cap.read()
            if ret:
                if not self.processing_started.is_set():
                    self.processing_started.set()
                
                try:
                    self.frame_queue.get_nowait()
                except:
                    pass
                self.frame_queue.put(frame)

    def find_droidcam_system_camera(self):
        for index in range(10):
            cap = cv2.VideoCapture(index)
            if cap.isOpened():
                ret, frame = cap.read()
                if ret and frame is not None:
                    h, w = frame.shape[:2]
                    
                    if w >= 640 and h >= 480:
                        cv2.imshow(f'Test Camera {index}', frame)
                        print(f"Camera {index}: {w}x{h} - Press 'y' for yes, any key for no")
                        key = cv2.waitKey(0)
                        cv2.destroyAllWindows()
                        
                        if key == ord('y'):
                            cap.release()
                            return index
                cap.release()
        
        return None

    def start_stream(self):
        self.camera_index = self.find_droidcam_system_camera()
        
        if self.camera_index is None:
            print("Phone camera not found")
            return
        
        self.cap = cv2.VideoCapture(self.camera_index)
        
        self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)      
        self.cap.set(cv2.CAP_PROP_FPS, 21)            
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)   
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)  
        
        # Start instant capture thread
        self.running = True
        capture_thread = threading.Thread(target=self.instant_frame_capture_thread, daemon=True)
        capture_thread.start()
        
        cv2.namedWindow('StreamSafe - Instant Processing', cv2.WINDOW_AUTOSIZE)
        
        # Feature toggles
        license_plate_protection = False
        address_number_protection = False
        street_sign_protection = False
        screen_protection = False
        document_protection = False
        
        print("Controls: 1=License Plates, 2=Address Numbers, 3=Street Signs, 4=Screens, 5=Documents, Q=Quit")
        
        try:
            while self.running:
                if not self.processing_started.is_set():
                    self.processing_started.wait(timeout=1.0)
                    continue
                
                try:
                    frame = self.frame_queue.get_nowait()
                except:
                    key = cv2.waitKey(1) & 0xFF
                    if key == ord('q'):
                        self.running = False
                    continue
                
                # Process frame with selected protections
                display_frame = frame.copy()
                if license_plate_protection:
                    display_frame = self.blur_license_plates(display_frame)
                if address_number_protection:
                    display_frame = self.blur_address_numbers(display_frame)
                if street_sign_protection:
                    display_frame = self.blur_street_signs(display_frame)
                if screen_protection:
                    display_frame = self.blur_screens(display_frame)
                if document_protection:
                    display_frame = self.blur_documents(display_frame)
                
                cv2.imshow('StreamSafe', display_frame)
                
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    self.running = False
                elif key == ord('1'):
                    license_plate_protection = not license_plate_protection
                    print(f"License plate protection {'enabled' if license_plate_protection else 'disabled'}")
                elif key == ord('2'):
                    address_number_protection = not address_number_protection
                    print(f"Address number protection {'enabled' if address_number_protection else 'disabled'}")
                elif key == ord('3'):
                    street_sign_protection = not street_sign_protection
                    print(f"Street sign protection {'enabled' if street_sign_protection else 'disabled'}")
                elif key == ord('4'):
                    screen_protection = not screen_protection
                    print(f"Screen protection {'enabled' if screen_protection else 'disabled'}")
                elif key == ord('5'):
                    document_protection = not document_protection
                    print(f"Document protection {'enabled' if document_protection else 'disabled'}")
        
        except KeyboardInterrupt:
            pass
        finally:
            self.running = False
            self.cap.release()
            cv2.destroyAllWindows()
# Usage
if __name__ == "__main__":
    print("StreamSafe")
    stream = StreamSafe()
    stream.start_stream()

StreamSafe
✅ License plate model loaded
Camera 0: 640x480 - Press 'y' for yes, any key for no
Camera 1: 640x480 - Press 'y' for yes, any key for no
Controls: 1=License Plates, 2=Address Numbers, 3=Street Signs, 4=Screens, 5=Documents, Q=Quit
License plate protection enabled
License plate protection disabled
Address number protection enabled
